In [ ]:
!pip3 install MNN opencv-python numpy matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/mnn

/content/drive/MyDrive/mnn


In [ ]:
!pip install -U pip
!pip install -U MNN

In [ ]:
# Copyright @ 2020 Alibaba. All rights reserved.
# Created by ruhuan on 2020.09.17
# Part of demo resource comes from MNN contributor, see below link FYI. 
# https://github.com/xindongzhang/MNN-APPLICATIONS
""" python demo usage about multi-input/multi-output MNN API """
from __future__ import print_function
import numpy as np
import cv2
import matplotlib.pyplot as plt
import MNN

In [ ]:

def inference():
    """ inference face detect using a specific picture
        
        the face detect network's backbone is mobile-ssd w/ two outputs tensors.
    """
    #create the MNN interpreter from model file
    interpreter = MNN.Interpreter("/content/drive/MyDrive/mnn/ssrnet.mnn")
    
    #create the MNN session by using MNN interpreter
    session = interpreter.createSession()
    
    #get the input dict from the session 
    inputs = interpreter.getSessionInputAll(session)
    
    #get the output dict from the session 
    outputs = interpreter.getSessionOutputAll(session)
    
    #get the input/output tensors from the dict 
    input0 = inputs['normalized_input_image_tensor']
    output0 = outputs['concat']
    output1 = outputs['concat_1']
    
    #cv2 read as bgr format
    ori_image = cv2.imread('/content/drive/MyDrive/mnn/r1.jpg')
    plt.title("input image")
    plt.imshow(ori_image[:,:,[2,1,0]])
    plt.show()
    
    #change to rgb format
    image = ori_image[..., ::-1]
    
    #resize to mobile_net tensor size
    image = cv2.resize(image, (224, 224))
    
    #preprocess it
    image = (image - 123.) / 58. 
   
    #as cv2 read shape is NHWC, Tensor's need is NCHW,transpose it
    image = image.transpose((2, 0, 1))
    
    #change numpy data type as np.float32 to match tensor's format
    image = image.astype(np.float32)
    
    #create temporary tensors for copy in/out 
    tmp_input0 = MNN.Tensor((1, 3, 224, 224), MNN.Halide_Type_Float,\
                    image, MNN.Tensor_DimensionType_Caffe)
    tmp_output0 = MNN.Tensor((1, 1014, 2), MNN.Halide_Type_Float,\
                    tuple(1 * 1014 * 2 *[1.0]), MNN.Tensor_DimensionType_Caffe)
    tmp_output1 = MNN.Tensor((1, 4, 1014, 1), MNN.Halide_Type_Float,\
                    tuple(1 * 4 * 1014 * 1 *[1.0]), MNN.Tensor_DimensionType_Caffe)
    
    #copy tensors values as inputs 
    input0.copyFrom(tmp_input0)
 
    
    #run session
    interpreter.runSession(session)
    
    #copy tensors value as outputs
    output0.copyToHostTensor(tmp_output0)
    output1.copyToHostTensor(tmp_output1)
    
    #get output data
    scores = tmp_output0.getData()
    boxes = tmp_output1.getData()
    
    #as the orignal tensors are [1, 1014, 2], [1, 4, 1014, 1], we reshape them
    scores = scores.reshape([2, 1014])
    boxes = boxes.reshape([4, 1014])

    print(scores,boxes)
    
    '''
    
    #contant values for anchors and scales
    face_prob_thresh = 0.3
    max_prob = 0.
    anchors = np.load("/content/drive/My Drive/MNN-tutorial/facedetection_tutorial/resource/anchors.npy")
    X_SCALE    = 10.0
    Y_SCALE    = 10.0
    H_SCALE    = 5.0
    W_SCALE    = 5.0
    
    #loop and find the face w/ most probability
    for i in range(1014):
        ycenter = boxes[0][i] / Y_SCALE * anchors[2][i] + anchors[0][i]
        xcenter = boxes[1][i] / X_SCALE * anchors[3][i] + anchors[1][i]
        h = np.exp(boxes[2][i] / H_SCALE) * anchors[2][i]
        w = np.exp(boxes[3][i] / W_SCALE) * anchors[3][i]
        ymin    = ( ycenter - h * 0.5 ) * ori_image.shape[0];
        xmin    = ( xcenter - w * 0.5 ) * ori_image.shape[1];
        ymax    = ( ycenter + h * 0.5 ) * ori_image.shape[0];
        xmax    = ( xcenter + w * 0.5 ) * ori_image.shape[1];
        nonface_prob = np.exp(scores[0][i])
        face_prob = np.exp(scores[1][i])
        ss = nonface_prob + face_prob
        nonface_prob /= ss
        face_prob /= ss
        if face_prob > face_prob_thresh and face_prob_thresh > max_prob:
            if xmin > 0 and ymin > 0 and xmax < ori_image.shape[1] and ymax < ori_image.shape[0]:
                max_prob = face_prob
                pt1 = (int(xmin), int(ymin))
                pt2 = (int(xmax), int(ymax))
    cv2.rectangle(ori_image, pt1, pt2, (255, 0, 0), 2)
    plt.title("result image")
    plt.imshow(ori_image[:,:,[2,1,0]])
    plt.show()    '''
    
if __name__ == "__main__":
    inference()